In [ ]:
import numpy as np
import shutil
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import dimod
from choi import choi
from chancellor import chancellor
from nuesslein1 import nuesslein1
import os
from nuesslein2 import nuesslein2
from solvers import sample_PT
import utils

options = {
    'nuesslein1': nuesslein1,
    'nuesslein2': nuesslein2,
    'chancellor': chancellor,
    'choi': choi
}

def convert(method: str, problem_path: str, output: str):
    V, C, formula = utils.read_formula(problem_path)
    method_func = options[method]
    formulation = method_func(formula, V)
    formulation.fillQ()
    problem = dimod.BQM.from_qubo(formulation.Q)
    problem.change_vartype('SPIN')
    biases, (row_inds, col_inds, vals), const =\
        problem.to_numpy_vectors(sort_labels=True)
    nzvals = np.count_nonzero(np.abs(vals) > 1e-10)
    print(const)
    print(nzvals, len(row_inds), len(col_inds))
    with open(output, 'w') as out:
        out.write(f'{problem.num_variables} '+
                  f'{problem.num_variables} {nzvals}\n')
        for i, w in enumerate(biases):
            out.write(f'{i} {w}\n')
        for i, j, w in zip(row_inds, col_inds, vals):
            if abs(w) < 1e-10:
                continue
            out.write(f'{i} {j} {w}\n')
        
    return problem, formula
problem_path = os.environ['GSET']+"/CUSTOM_SAT/cust-u50-212/cust-u50-02.cnf"
problem, formula = convert('nuesslein2', problem_path, 'cust-u50-02_n2.ising')
args = {
    "num_sweeps": 10000,
    "num_reads": 50,
    "num_replicas": -1,
    "min_temp": 0.3,
    "max_temp": 1.5,
    "update_strategy": "sequential",
    "initialize_strategy": "random",
    "init_energies": -1,
    "recompute_energy": 0,
    "sort_output_temps": 0,
    "return_dataframe": 1,
    "parallel": 1,
    "use_pt": 1,
    "send_background": 0,
    "verbose": 1,
    "get_part_fun": 0,
    "beta0": 0
}
# result = sample_PT(problem, **args)


/home/matt/miniconda3/envs/quantum/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


97.5
880 882 882


In [4]:
from nuesslein2 import nuesslein2
from pathlib import Path
import os
from tqdm import tqdm
from utils import read_formula, check_solution
n = [20, 50, 75, 100, 125, 150, 175, 200]
m = [91, 218, 325, 430, 538, 645, 753, 860]
basedir = os.environ['GSET'] + '/SATLIB_GRAPHS/uf{n}-{m}/uf{n}-0{i}.cnf'
for n, m in tqdm(zip(n, m)):
    for i in range(1,51):
        test_problem = basedir.format(n=n, m=m, i=i)
        newname = Path(test_problem).stem
        n,_, clauses = read_formula(test_problem)
        test = nuesslein2(clauses, n)
        # print(test_problem)
        # print(np.min([len(c) for c in clauses]))
        test.fillQ()
        test.write(f'cust_reduced_nusslein/{newname}.ising')

8it [00:04,  1.96it/s]
